In [2]:
#standard imports
import os, time, json, datetime, pytz
from tqdm.auto import tqdm
from typing import List, Union, Dict
from re_sent_splitter import split_into_sentences
from pathlib import Path
import pathlib
from multiprocessing import Pool
import string
import numpy as np
import pandas as pd

import torch.nn as nn
import torch
from transformers import BertTokenizer, DataCollatorForWholeWordMask, DataCollatorForLanguageModeling
import pandas as pd
import numpy as np
from sklearn import model_selection
from transformers import AdamW, get_linear_schedule_with_warmup
from scipy import stats

#distributed imports
import torch
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP, DataParallel
from torch.utils.data import DistributedSampler, DataLoader
import deepspeed

#tokenizers and datasets
from datasets import load_dataset
from tokenizers import BertWordPieceTokenizer 
from tokenizers.processors import TemplateProcessing
import tokenizers

#transformer imports
from transformers import BertTokenizer, DataCollatorForWholeWordMask, DataCollatorForLanguageModeling, BertConfig
from transformers import BertModel,BertForMaskedLM, BertConfig, AdamW, TrainingArguments, Trainer
from transformers import pipeline

In [3]:
model_path = '/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/test_save_pretrained/model-trained-14000.pt/'

In [4]:
alternative_tokenizer = BertTokenizer.from_pretrained('../Preprocessing/Tokenization/wp-vocab-30500-vocab.txt')

tokenizer = BertWordPieceTokenizer('../Preprocessing/Tokenization/wp-vocab-30500-vocab.txt', strip_accents=True, lowercase=True)
tokenizer.enable_truncation(max_length=512)
tokenizer.enable_padding()
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
        ("[MASK]", tokenizer.token_to_id("[MASK]"))
    ],
)

/anaconda/envs/py38_pytorch/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1658: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [5]:
class TrainedBERT(nn.Module):
    def __init__(self):
        super(TrainedBERT, self).__init__()
        self.bert = BertModel.from_pretrained(model_path)
        self.out = nn.Linear(768, 3)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, ids, mask, token_type_ids):
        _, output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        output = self.out(output)
        return self.softmax(output)

In [10]:
class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = BertModel(config=BertConfig(vocab_size=30500))
        self.out = nn.Linear(768, 3)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, ids, mask, token_type_ids):
        _, output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        output = self.out(output)
        return self.softmax(output)

class BERTDatasetTraining:
    def __init__(self, question, context, targets, tokenizer, max_len):
        self.question = question
        self.context = context
        self.targets = targets
        
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.question)
    
    def __getitem__(self, item):
        question= str(self.question[item])
        context = str(self.context[item])

        inputs = self.tokenizer.encode_plus(
            question,
            context,
            add_special_tokens=True
        )

        ids = inputs['input_ids']
        token_type_ids = inputs['token_type_ids']
        mask = inputs['attention_mask']
        
        padding_len = self.max_len - len(ids)
        
        ids = ids[:self.max_len] + ([0] * padding_len) 
        token_type_ids = token_type_ids[:self.max_len] + ([0] * padding_len)
        mask = mask[:self.max_len] + ([0] * padding_len)
        

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[item], dtype=torch.long)
        }


def loss_fn(outputs, targets):
    return nn.CrossEntropyLoss()(outputs, targets)


def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    for bi, d in enumerate(data_loader):
        ids = d['ids']
        mask = d['mask']
        token_type_ids = d['token_type_ids']
        targets = d['targets']
        
        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.long)
        
        optimizer.zero_grad()
        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        loss.sum().backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        if bi % 50 == 0:
            print(f'bi={bi}, loss={loss}')

            
def eval_loop_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    for bi, d in enumerate(data_loader):
        with torch.no_grad():
          ids = d['ids'].to(device, dtype=torch.long)
          mask = d['mask'].to(device, dtype=torch.long)
          token_type_ids = d['token_type_ids'].to(device, dtype=torch.long)
          targets = d['targets'].to(device, dtype=torch.long)
        
          outputs = model(ids, mask, token_type_ids)
          #loss = loss_fn(outputs, targets)
        
          fin_targets.append(targets.cpu().detach().numpy())
          fin_outputs.append(outputs.cpu().detach().numpy())

    return np.vstack(fin_outputs), np.hstack(fin_targets)


def run():
    MAX_LEN = 512
    TRAIN_BATCH_SIZE = 8
    EPOCHS = 4
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    
    df_train = pd.read_json('./data/pqal_fold0/train_set.json', orient='index')
    df_valid = pd.read_json('./data/pqal_fold0/dev_set.json', orient='index')
    

    target_cols = 'final_decision'
    train_targets_str = df_train[target_cols].values
    valid_targets_str = df_valid[target_cols].values
    
    # Transformer the target col to numeric values
    target_dict = {'yes': 0, 'maybe': 1, 'no': 2}
    train_targets = [target_dict[key] for key in train_targets_str]
    valid_targets = [target_dict[key] for key in valid_targets_str]
    

    tokenizer = alternative_tokenizer 
    #model = TrainedBERT()
    model = BERTBaseUncased()
    model = DataParallel(model)
    model.to(device)

    train_dataset = BERTDatasetTraining(
        question=df_train.QUESTION.values,
        context=df_train.CONTEXTS.values,
        targets=train_targets,
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        shuffle=True
    )
    
    valid_dataset = BERTDatasetTraining(
        question=df_valid.QUESTION.values,
        context=df_valid.CONTEXTS.values,
        targets=valid_targets,
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )
    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=4,
        shuffle=True,
        drop_last=True
    )
    
    optimizer = AdamW(model.parameters(), lr=2e-5)
    num_training_steps = int(len(train_dataset) / TRAIN_BATCH_SIZE * EPOCHS)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )
    
    for epoch in range(EPOCHS):
        train_loop_fn(train_data_loader, model, optimizer, device, scheduler)
        output, target = eval_loop_fn(valid_data_loader, model, device)
        acc = (output.argmax(1) == target).sum() / len(target)
        print(f'epoch: {epoch}, acc: {acc}')
    
    # Inference
    TEST_BATCH_SIZE = 8
    df_test = pd.read_json('./data/test_set.json', orient='index')
    test_targets_str = df_test[target_cols].values
    test_targets = [target_dict[key] for key in test_targets_str]


    test_dataset = BERTDatasetTraining(
        question=df_test.QUESTION.values,
        context=df_test.CONTEXTS.values,
        targets=test_targets, # This is not used
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )
    test_data_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=TEST_BATCH_SIZE,
        shuffle=False
    )

    predictions = []
    model.eval()

    for bi, d in enumerate(test_data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        
        with torch.no_grad():
            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids).cpu().detach().numpy()
            predictions.append(outputs.argmax(1))
    pmids = df_test.index

    return pmids, np.hstack(predictions)

In [11]:
pmids, predictions = run()

/anaconda/envs/py38_pytorch/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


bi=0, loss=1.0881009101867676
bi=50, loss=1.0495349168777466
epoch: 0, acc: 0.5416666666666666
bi=0, loss=0.8044472932815552
bi=50, loss=0.9325860142707825
epoch: 1, acc: 0.5416666666666666
bi=0, loss=1.0519938468933105
bi=50, loss=0.8075883984565735
epoch: 2, acc: 0.5416666666666666
bi=0, loss=1.1462160348892212
bi=50, loss=1.0988537073135376
epoch: 3, acc: 0.5625


In [12]:
import json
from sklearn.metrics import accuracy_score, f1_score
ground_truth = pd.read_json('./data/test_ground_truth.json', orient='index')

In [13]:
truth_list = ground_truth[0].tolist()
target_dict = {'yes': 0, 'maybe': 1, 'no': 2}
truth_list = [target_dict[key] for key in truth_list]

In [14]:
(truth_list == predictions).sum() / len(predictions)

0.552

In [67]:
!nvidia-smi

Fri Mar 25 20:58:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.54       Driver Version: 510.54       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000001:00:00.0 Off |                  Off |
| N/A   31C    P0    35W / 250W |  16151MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000002:00:00.0 Off |                  Off |
| N/A   

In [68]:
torch.cuda.empty_cache()

In [1]:
!nvidia-smi

Fri Mar 25 20:58:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.54       Driver Version: 510.54       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000001:00:00.0 Off |                  Off |
| N/A   30C    P0    35W / 250W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000002:00:00.0 Off |                  Off |
| N/A   

In [9]:
cd ../Modeling/FineTuning/

/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/FineTuning


In [18]:
pd.read_json('data/test_ground_truth.json', orient='index').value_counts().sum()

500

In [19]:
276/500

0.552